# Fashion MNIST

## Data Loading & Processing

In [21]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
import tqdm

In [3]:
train_dataset = datasets.FashionMNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)

In [5]:
images, _ = next(iter(train_loader))
mean = images.mean().item()
std = images.std().item()
print(mean, std)

0.28604060411453247 0.3530242443084717


In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean,), (std,))
])

In [9]:
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [15]:
class fashionMNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 784),
            nn.ReLU(),
            nn.Linear(784, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [25]:
%%time

model = fashionMNISTClassifier()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in tqdm.tqdm(range(10)):
    for X, y in train_loader:
        #forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        #backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|███████████████████████████████████████████| 10/10 [00:34<00:00,  3.41s/it]

CPU times: user 31.7 s, sys: 2.91 s, total: 34.6 s
Wall time: 34.1 s


## Evaluation 

In [28]:
model.eval()

correct = 0
total = 0

In [32]:
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        pred = outputs.argmax(dim=1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Test accuracy percentage: {accuracy * 100:.2f}%")

Test accuracy percentage: 79.99%
